# HO-FMN: Hyperparameter Optimization for Fast Minimum-Norm Attacks

In [1]:
import torch
from torch.utils.data import DataLoader

from src.ho_fmn.ho_fmn import HOFMN
from src.attacks.fmn import FMN

from src.utils.model_data import load_data
from src.utils.model_data import print_models_info
from src.utils.fmn_config import print_fmn_configs
from src.utils.show_image import show_image

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

Execute the next cell to see the list of all the currently available models, losses, optimizers and schedulers on which HO-FMN has been tested.

In [2]:
print_models_info()
print_fmn_configs()

Model ID	Name
0			Wang2023Better_WRN-70-16
1			Wang2023Better_WRN-28-10
2			Gowal2021Improving_70_16_ddpm_100m
3			Rebuffi2021Fixing_106_16_cutmix_ddpm
4			Gowal2021Improving_28_10_ddpm_100m
5			Pang2022Robustness_WRN70_16
6			Sehwag2021Proxy_ResNest152
7			Pang2022Robustness_WRN28_10
8			Gowal2021Improving_R18_ddpm_100m
9			Rade2021Helper_R18_ddpm
10			Sehwag2021Proxy_R18
11			Rebuffi2021Fixing_R18_ddpm

Losses:
	LL
	DLR
	CE
Optimizers:
	SGD
	Adam
	Adamax
Schedulers:
	CALR
	RLROP


Let's choose a lightweight model for this demo (e.g. model 8, Gowal2021Improving_R18_ddpm_100m). When can compute its performance in classifying clean (unperturbed) samples.

In [3]:
model_id = 8
batch_size = 32

model, dataset, model_name, _ = load_data(model_id=model_id)
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False
)

Files already downloaded and verified


## Compute clean accuracy
Testing the model's performance with clean (unperturbed) samples

In [4]:
total, correct = 0.0, 0.0
for b, (inputs, labels) in enumerate(dataloader):
    inputs, labels = inputs.to(device), labels.to(device)
    pred = model(inputs).argmax(1)
    total += labels.shape[0]
    correct += (pred == labels).sum()
    if total >= 1000:
        break

print(f"Model name: {model_name}")
print(f"Clean accuracy: {correct/total:.4f}")

Model name: Gowal2021Improving_R18_ddpm_100m
Clean accuracy: 0.8828


## HO-FMN in action
We firstly instantiate the HO-FMN tuner object, it will take the model, a configuration in terms of Loss-Optimizer-Scheduler as input; as well as the batch size, the number of HO trials etc.


In [5]:
loss = 'DLR'        # Difference of Logits Ratio
optimizer = 'SGD'   # Stochastic Gradient Descent
scheduler = 'CALR'  # Cosine Annealing Learning Rate

batch_size = 32     # Size of a single batch
steps = 100         # The number of FMN's iterations
trials = 10         # Number of HO optimization trials

'''
HO-FMN performs N = trials number of attacks; the final number of samples that are used for the optimization
are TUNING_SAMPLES = trials * batch_size (e.g., 64*32 = 2048 tuning samples).
'''

ho_fmn = HOFMN(
    model=model,
    dataloader=dataloader,
    loss=loss,
    optimizer=optimizer,
    scheduler=scheduler,
    steps=steps,
    trials=trials,
    verbose=True,
    device=device
)

Then we start the HO tuning process by calling the tune() method. The tuner, after completing the process, will return a dictionary containing all the best parameters found.

In [6]:
best_parameters = ho_fmn.tune()
print(f"Best parameters:\n{best_parameters}")

[INFO 02-20 14:32:46] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 02-20 14:32:46] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='lr', parameter_type=FLOAT, range=[0.03137254901960784, 10.0], log_scale=True), RangeParameter(name='momentum', parameter_type=FLOAT, range=[0.0, 0.9]), RangeParameter(name='weight_decay', parameter_type=FLOAT, range=[0.01, 1.0]), RangeParameter(name='dampening', parameter_type=FLOAT, range=[0.0, 0.2]), FixedParameter(name='T_max', parameter_type=INT, value=100), FixedParameter(name='eta_min', parameter_type=INT, value=0), FixedParameter(name='last_epoch', parameter_type=INT, value=-1)], parameter_constraints=[]).
[INFO 02-20 14:32:46] ax.modelbridge.dispatch_utils: Using Models.BOTORCH_MODULAR since there are more ordered parameters than there are 

	[Tuning] Creating the Ax client and experiment...
	[Tuning] Starting the Hyperparameters Optimization...
	[Tuning] Running trial 0


ValueError: only one element tensors can be converted to Python scalars

## Some insights on the HO process
We can use some of the available functions from the Ax Tuning Framework to see the results of the Bayesian Optimization. We plot the interactable contour, on which we see the median and the standard error for the objective metric of the HO; in addition, we plot the slice, or the representative plot for the BO regressor using the learning rate as parameter.

In [ ]:
from ax.plot.contour import interact_contour
from ax.plot.slice import plot_slice
from ax.utils.notebook.plotting import init_notebook_plotting, render

init_notebook_plotting()

model = ho_fmn.ax_client.generation_strategy.model
render(interact_contour(model=model, metric_name="distance"))
render(plot_slice(model=model, param_name='lr', metric_name="distance"))

## Perform FMN attack using the best configuration
From the previous tuning we got the best configuration of hyperparameters for the optmizer and scheduler.
We run the attack using this configuration.

In [ ]:
tuning_bs = batch_size
tuning_trials = trials

batch_size = 125
n_batches = 8   # TEST_SAMPLES = batch_size * n_batches (e.g., 1000 samples = 125*8)

tuning_samples = tuning_bs*tuning_trials
subset_indices = list(range(tuning_samples, tuning_samples + batch_size*n_batches))
dataset_frac = torch.utils.data.Subset(dataset, subset_indices)

print(f"Samples: {len(subset_indices)}")

dataloader = DataLoader(
    dataset=dataset_frac,
    batch_size=batch_size,
    shuffle=False
)

# Extract the optimizer and scheduler config from the best params dictionary
optimizer_config, scheduler_config = ho_fmn.parametrization_to_configs(best_parameters)

fmn = FMN(
    model=model,
    steps=steps,
    loss=loss,
    optimizer=optimizer,
    scheduler=scheduler,
    optimizer_config=optimizer_config,
    scheduler_config=scheduler_config
)

# Testing on a single batch
images, labels = next(iter(dataloader))

best_adv = fmn.forward(images, labels)

## Show original and perturbed images

In [ ]:
preds = model(best_adv).argmax(1)

img_shape = (3, 32, 32)
show_image('HO-FMN', images, best_adv, preds, labels, n_display=8, img_shape=img_shape)